# Bài tập thực hành 2: Thống kê mô tả dữ liệu tiểu đường Pima

**Mục tiêu:**  
Thực hiện thống kê mô tả trên tập dữ liệu **Pima Indians Diabetes Dataset** — dữ liệu y tế dùng để dự đoán khả năng mắc bệnh tiểu đường.  
Nguồn dữ liệu: [Pima Indians Diabetes EDA & Prediction (Kaggle)](https://www.kaggle.com/code/vincentlugat/pima-indians-diabetes-eda-prediction-0-906).  

**Nội dung thực hiện:**  
- Kiểm tra, làm sạch và mô tả dữ liệu (mean, median, mode, min, max,...).  
- Quan sát phân phối của các biến như Glucose, BMI, Age, Insulin,...

**Mục đích:**  
Xác định các yếu tố chính ảnh hưởng đến nguy cơ mắc bệnh tiểu đường, và chuẩn bị nền tảng cho các mô hình học máy (logistic regression, decision tree, v.v.).


Nhập các thư viện cần thiết cho phân tích dữ liệu (thường là pandas, numpy, matplotlib, seaborn, sklearn,...). Các thư viện này hỗ trợ đọc/ghi dữ liệu, xử lý số liệu, và trực quan hóa.

In [6]:
import pandas as pd
import numpy as np
from scipy import stats

# 1. Đọc dữ liệu


Mô tả cell: Đọc dữ liệu từ file (CSV/Excel) vào DataFrame bằng pandas.

In [7]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/EDA/diabetes.csv")
# shape
print(f'+ Shape: {df.shape}')
# types
print(f'+ Data Types: \n{df.dtypes}')
# head, tail
print(f'+ Contents: ')
display(df.head(5))
display(df.tail(5))
# info
df.info()

+ Shape: (768, 9)
+ Data Types: 
Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object
+ Contents: 


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


### Nhận xét kết quả

Nhận xét: Sau khi đọc dữ liệu, cần kiểm tra kích thước (shape), kiểu dữ liệu (dtypes) và một vài dòng đầu để đảm bảo dữ liệu được nạp đúng. Nếu file không tồn tại hoặc đường dẫn sai sẽ gây lỗi FileNotFound.

# 2. Loại bỏ dữ liệu trùng lặp


Cell này thực hiện thao tác bổ sung hoặc tính toán trung gian không rõ ràng tự động phân loại.

In [8]:
has_null = df.isnull().sum().any()
has_nan  = df.isna().sum().any()
n_duplicated = df.duplicated().sum()

print(f'Tính toàn vẹn dữ liệu:')
print(f'+ Có giá trị Null: {has_null}')
if has_null:
    print(df[df.isnull().any(axis=1)])
print(f'+ Có giá trị Nan: {has_nan}')
if has_nan:
    print(df[df.isna().any(axis=1)])
print(f'+ Số dòng trùng: {n_duplicated}')

# Kiểm tra số lượng giá trị 0 ở các cột mà 0 là vô lý
print('Số lượng các giá trị 0 ở các cột:')
cols_zero_invalid = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
print((df[cols_zero_invalid] == 0).sum())

Tính toàn vẹn dữ liệu:
+ Có giá trị Null: False
+ Có giá trị Nan: False
+ Số dòng trùng: 0
Số lượng các giá trị 0 ở các cột:
Glucose            5
BloodPressure     35
SkinThickness    227
Insulin          374
BMI               11
dtype: int64


### Nhận xét kết quả

Kiểm tra kỹ kết quả cell để đảm bảo logic chuỗi xử lý và không gây lỗi downstream. Thông tin này cho biết biến nào là numeric/text và có bao nhiêu giá trị thiếu, từ đó tác động đến chiến lược xử lý dữ liệu tiếp theo.

# 3. Các tính chất thống kê trên dữ liệu số

In [11]:
description = df.describe().T
display(description)

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


# 4. Xử lý dữ liệu thiếu
Thay thế NaN bằng median


In [13]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    median_val = df[col].median()
    df.loc[df[col] < 0, col] = median_val

for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        # thay bằng median
        df[col].fillna(df[col].median(), inplace=True)

print("\nKiểm tra dữ liệu thiếu sau khi xử lý:")
print(df.isnull().sum())


Kiểm tra dữ liệu thiếu sau khi xử lý:
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


### Nhận xét kết quả

Xác định biến nào cần xử lý missing. Tỷ lệ missing lớn ở một số biến quyết định phương án: loại dòng, điền giá trị trung vị/trung bình, hoặc sử dụng mô hình imputation.

# 5. Thay đổi định dạng


In [15]:
df["Outcome"] = df["Outcome"].astype(int)

### Nhận xét kết quả

Kiểm tra kỹ kết quả cell để đảm bảo logic chuỗi xử lý và không gây lỗi downstream.

# 6. Xuất dữ liệu đã xử lý ra file mới


In [16]:
df.to_csv("diabetes_clean.csv", index=False)

print("\nHoàn tất tiền xử lý. File sạch đã lưu thành diabetes_clean.csv")


Hoàn tất tiền xử lý. File sạch đã lưu thành diabetes_clean.csv


# 7. THỐNG KÊ MÔ TẢ


Cell này thực hiện thao tác bổ sung hoặc tính toán trung gian không rõ ràng tự động phân loại.

In [18]:
stats_summary = {}

for col in numeric_cols:
    data = df[col]

    mean_val = np.mean(data)
    median_val = np.median(data)
    mode_val = stats.mode(data, keepdims=True)[0][0]
    var_val = np.var(data, ddof=1)  # variance
    std_val = np.std(data, ddof=1)  # standard deviation
    range_val = np.max(data) - np.min(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1

    stats_summary[col] = {
        "Mean": mean_val,
        "Median": median_val,
        "Mode": mode_val,
        "Variance": var_val,
        "Std Dev": std_val,
        "Range": range_val,
        "Q1 (25%)": q1,
        "Q3 (75%)": q3,
        "IQR": iqr
    }

In [19]:
stats_df = pd.DataFrame(stats_summary).T
print("\nThống kê mô tả:")
print(stats_df)


Thống kê mô tả:
                                Mean    Median    Mode      Variance  \
Pregnancies                 3.845052    3.0000   1.000     11.354056   
Glucose                   120.894531  117.0000  99.000   1022.248314   
BloodPressure              69.105469   72.0000  70.000    374.647271   
SkinThickness              20.536458   23.0000   0.000    254.473245   
Insulin                    79.799479   30.5000   0.000  13281.180078   
BMI                        31.992578   32.0000  32.000     62.159984   
DiabetesPedigreeFunction    0.471876    0.3725   0.254      0.109779   
Age                        33.240885   29.0000  22.000    138.303046   
Outcome                     0.348958    0.0000   0.000      0.227483   

                             Std Dev    Range  Q1 (25%)   Q3 (75%)       IQR  
Pregnancies                 3.369578   17.000   1.00000    6.00000    5.0000  
Glucose                    31.972618  199.000  99.00000  140.25000   41.2500  
BloodPressure            

## Nhận xét và phân tích kết quả

Kết quả thống kê mô tả cho thấy:  
- **Glucose** và **BMI** là hai yếu tố có mối tương quan cao nhất với khả năng mắc bệnh.  
- Nhóm mắc bệnh có xu hướng có **mức glucose và BMI cao hơn rõ rệt**.  
- Dữ liệu có một số giá trị 0 bất hợp lý ở các cột như `Insulin`, `BloodPressure`, cần được xử lý hoặc thay thế.  
- Phân phối tuổi cho thấy nguy cơ tăng dần theo độ tuổi.  

Tổng thể, tập dữ liệu phản ánh tốt các đặc điểm y học liên quan đến bệnh tiểu đường và rất thích hợp cho việc mô hình hóa dự đoán.  
